## A Simple Pair Trading Strategy


Builds a simple pair trading strategy to show an example of a strategy that relies on relationships between multiple contracts.

1.  We will buy AAPL and sell NVDA when the price ratio AAPL / NVDA is more than 1 standard deviation lower than its 1 day simple moving average.  
2.  We will buy NVDA and sell AAPL when the price ratio AAPL / NVDA is more than 1 standard deviation higher than its 1 day simple moving average.
3.  We will exit when the price ratio is less than +/- 0.5 standard deviations away from its simple moving average
4.  We will size the trades by allocating 10% of our capital to each trade.

In [3]:
%%checkall
import pandas as pd
import pyqstrat as pq

_logger = pq.get_child_logger(__name__)


if __name__ == '__main__':

    pq.set_defaults()  # Set some display defaults to make dataframes and plots easier to look at

    # read 1 minute price bars
    aapl = pd.read_csv('./data//AAPL.csv.gz', parse_dates=['timestamp'])[['timestamp', 'c']]
    nvda = pd.read_csv('./data/NVDA.csv.gz', parse_dates=['timestamp'])[['timestamp', 'c']]
    prices = pd.merge(aapl, nvda, on=['timestamp'], how='outer', suffixes=['_aapl', '_nvda'])
    prices['ratio'] = prices.c_aapl / prices.c_nvda
    
    # compute zscore over rolling 1 day 
    r = pd.Series(prices.ratio).rolling(window=int(60 * 6.5))  # 1 day * 6.5 trading hours / day * 60 minutes / hour
    mean, std = r.mean(), r.std(ddof=0)
    zscore = (prices.ratio - mean) / std
    prices['zscore'] = zscore

    strat_builder = pq.StrategyBuilder(prices)
    aapl_contract = strat_builder.add_contract('AAPL')
    nvda_contract = strat_builder.add_contract('NVDA')

    price_dict = {'AAPL': {aapl.timestamp.values[i].astype('M8[m]'): aapl.c.values[i] for i in range(len(aapl))},
                  'NVDA': {nvda.timestamp.values[i].astype('M8[m]'): nvda.c.values[i] for i in range(len(nvda))}}
    price_func = pq.PriceFuncDict(price_dict)
    strat_builder.set_price_function(price_func)

    prices['enter_long'] = (prices.zscore > 1)
    prices['enter_short'] = (prices.zscore < -1)
    prices['exit_long'] = ((prices.zscore > 0.5) * (prices.zscore < 1))
    prices['exit_short'] = ((prices.zscore > 0.5) * (prices.zscore < 1))
    

    long_entry_rule = pq.PercentOfEquityTradingRule('LONG', price_func=price_func, long=True)
    short_entry_rule = pq.PercentOfEquityTradingRule('SHORT', price_func=price_func, long=False) 
    
    aapl_cg = aapl_contract.contract_group
    nvda_cg = nvda_contract.contract_group

    # when the enter_long signal fires we want to go long aapl and short nvda and when enter_short fires
    # we want to go short aapl, long nvda    
    strat_builder.add_series_rule('enter_long', long_entry_rule, 'zero', 'long_aapl', contract_groups=[aapl_cg])
    strat_builder.add_series_rule('enter_short', short_entry_rule, 'zero', 'short_aapl', contract_groups=[aapl_cg])

    strat_builder.add_series_rule('enter_long', short_entry_rule, 'zero', 'long_nvda', contract_groups=[nvda_cg])
    strat_builder.add_series_rule('enter_short', long_entry_rule, 'zero', 'short_nvda', contract_groups=[nvda_cg])
    
    strat_builder.add_series_rule('exit_long', pq.ClosePositionExitRule('EXIT_LONG', price_func), 'positive')
    strat_builder.add_series_rule('exit_short', pq.ClosePositionExitRule('EXIT_SHORT', price_func), 'negative')

    strategy = strat_builder()
    strategy.run()

running typecheck
Success: no issues found in 1 source file

running flake8
stdin:39:5: E303 too many blank lines (2)

[2023-10-23 20:31:30.905 __call__] TRADE: 2023-01-04T09:32 AAPL  2023-01-04 09:32:00 qty: -395 prc: 126.7   order: AAPL 2023-01-04 09:31:00 qty: -395 SHORT  OrderStatus.OPEN 
[2023-10-23 20:31:30.906 __call__] TRADE: 2023-01-04T09:32 NVDA  2023-01-04 09:32:00 qty: -343 prc: 145.44   order: NVDA 2023-01-04 09:31:00 qty: -343 SHORT  OrderStatus.OPEN 
[2023-10-23 20:31:30.906 __call__] TRADE: 2023-01-04T09:32 AAPL  2023-01-04 09:32:00 qty: 395 prc: 126.7   order: AAPL 2023-01-04 09:31:00 qty: 395 LONG  OrderStatus.OPEN 
[2023-10-23 20:31:30.906 __call__] TRADE: 2023-01-04T09:32 NVDA  2023-01-04 09:32:00 qty: 343 prc: 145.44   order: NVDA 2023-01-04 09:31:00 qty: 343 LONG  OrderStatus.OPEN 
[2023-10-23 20:31:30.907 __call__] TRADE: 2023-01-04T09:33 AAPL  2023-01-04 09:33:00 qty: -394 prc: 126.32   order: AAPL 2023-01-04 09:32:00 qty: -394 SHORT  OrderStatus.OPEN 
[2023-10-

[2023-10-23 20:31:30.922 __call__] TRADE: 2023-01-04T09:43 NVDA  2023-01-04 09:43:00 qty: 340 prc: 146.11   order: NVDA 2023-01-04 09:42:00 qty: 340 LONG  OrderStatus.OPEN 
[2023-10-23 20:31:30.923 __call__] TRADE: 2023-01-04T09:44 AAPL  2023-01-04 09:44:00 qty: -395 prc: 126.164   order: AAPL 2023-01-04 09:43:00 qty: -395 SHORT  OrderStatus.OPEN 
[2023-10-23 20:31:30.923 __call__] TRADE: 2023-01-04T09:44 NVDA  2023-01-04 09:44:00 qty: -342 prc: 145.266   order: NVDA 2023-01-04 09:43:00 qty: -342 SHORT  OrderStatus.OPEN 
[2023-10-23 20:31:30.923 __call__] TRADE: 2023-01-04T09:44 AAPL  2023-01-04 09:44:00 qty: 395 prc: 126.164   order: AAPL 2023-01-04 09:43:00 qty: 395 LONG  OrderStatus.OPEN 
[2023-10-23 20:31:30.924 __call__] TRADE: 2023-01-04T09:44 NVDA  2023-01-04 09:44:00 qty: 342 prc: 145.266   order: NVDA 2023-01-04 09:43:00 qty: 342 LONG  OrderStatus.OPEN 
[2023-10-23 20:31:30.924 __call__] TRADE: 2023-01-04T09:45 AAPL  2023-01-04 09:45:00 qty: -396 prc: 126.17   order: AAPL 2023

[2023-10-23 20:31:30.940 __call__] TRADE: 2023-01-04T09:55 AAPL  2023-01-04 09:55:00 qty: 396 prc: 126.26   order: AAPL 2023-01-04 09:54:00 qty: 396 LONG  OrderStatus.OPEN 
[2023-10-23 20:31:30.941 __call__] TRADE: 2023-01-04T09:55 NVDA  2023-01-04 09:55:00 qty: 344 prc: 145   order: NVDA 2023-01-04 09:54:00 qty: 344 LONG  OrderStatus.OPEN 
[2023-10-23 20:31:30.941 __call__] TRADE: 2023-01-04T09:56 AAPL  2023-01-04 09:56:00 qty: -396 prc: 126.29   order: AAPL 2023-01-04 09:55:00 qty: -396 SHORT  OrderStatus.OPEN 
[2023-10-23 20:31:30.942 __call__] TRADE: 2023-01-04T09:56 NVDA  2023-01-04 09:56:00 qty: -344 prc: 144.982   order: NVDA 2023-01-04 09:55:00 qty: -344 SHORT  OrderStatus.OPEN 
[2023-10-23 20:31:30.942 __call__] TRADE: 2023-01-04T09:56 AAPL  2023-01-04 09:56:00 qty: 396 prc: 126.29   order: AAPL 2023-01-04 09:55:00 qty: 396 LONG  OrderStatus.OPEN 
[2023-10-23 20:31:30.942 __call__] TRADE: 2023-01-04T09:56 NVDA  2023-01-04 09:56:00 qty: 344 prc: 144.982   order: NVDA 2023-01-04

[2023-10-23 20:31:30.957 __call__] TRADE: 2023-01-04T10:39 NVDA  2023-01-04 10:39:00 qty: 349 prc: 143.04   order: NVDA 2023-01-04 10:38:00 qty: 349 LONG  OrderStatus.OPEN 
[2023-10-23 20:31:30.958 __call__] TRADE: 2023-01-04T10:39 AAPL  2023-01-04 10:39:00 qty: -396 prc: 126   order: AAPL 2023-01-04 10:38:00 qty: -396 SHORT  OrderStatus.OPEN 
[2023-10-23 20:31:30.958 __call__] TRADE: 2023-01-04T10:39 NVDA  2023-01-04 10:39:00 qty: -349 prc: 143.04   order: NVDA 2023-01-04 10:38:00 qty: -349 SHORT  OrderStatus.OPEN 
[2023-10-23 20:31:30.959 __call__] TRADE: 2023-01-04T10:40 AAPL  2023-01-04 10:40:00 qty: 396 prc: 126.135   order: AAPL 2023-01-04 10:39:00 qty: 396 LONG  OrderStatus.OPEN 
[2023-10-23 20:31:30.959 __call__] TRADE: 2023-01-04T10:40 NVDA  2023-01-04 10:40:00 qty: 349 prc: 142.97   order: NVDA 2023-01-04 10:39:00 qty: 349 LONG  OrderStatus.OPEN 
[2023-10-23 20:31:30.959 __call__] TRADE: 2023-01-04T10:40 AAPL  2023-01-04 10:40:00 qty: -396 prc: 126.135   order: AAPL 2023-01-0

[2023-10-23 20:31:30.974 __call__] TRADE: 2023-01-04T10:52 AAPL  2023-01-04 10:52:00 qty: 394 prc: 126.98   order: AAPL 2023-01-04 10:51:00 qty: 394 LONG  OrderStatus.OPEN 
[2023-10-23 20:31:30.974 __call__] TRADE: 2023-01-04T10:52 NVDA  2023-01-04 10:52:00 qty: 347 prc: 144.02   order: NVDA 2023-01-04 10:51:00 qty: 347 LONG  OrderStatus.OPEN 
[2023-10-23 20:31:30.975 __call__] TRADE: 2023-01-04T10:52 AAPL  2023-01-04 10:52:00 qty: -394 prc: 126.98   order: AAPL 2023-01-04 10:51:00 qty: -394 SHORT  OrderStatus.OPEN 
[2023-10-23 20:31:30.975 __call__] TRADE: 2023-01-04T10:52 NVDA  2023-01-04 10:52:00 qty: -347 prc: 144.02   order: NVDA 2023-01-04 10:51:00 qty: -347 SHORT  OrderStatus.OPEN 
[2023-10-23 20:31:30.976 __call__] TRADE: 2023-01-04T10:53 AAPL  2023-01-04 10:53:00 qty: 393 prc: 126.84   order: AAPL 2023-01-04 10:52:00 qty: 393 LONG  OrderStatus.OPEN 
[2023-10-23 20:31:30.976 __call__] TRADE: 2023-01-04T10:53 NVDA  2023-01-04 10:53:00 qty: 347 prc: 143.96   order: NVDA 2023-01-0

...

[2023-10-23 20:31:39.163 __call__] TRADE: 2023-01-27T14:55 NVDA  2023-01-27 14:55:00 qty: 243 prc: 205   order: NVDA 2023-01-27 14:54:00 qty: 243 LONG  OrderStatus.OPEN 
[2023-10-23 20:31:39.164 __call__] TRADE: 2023-01-27T14:56 AAPL  2023-01-27 14:56:00 qty: -340 prc: 146.98   order: AAPL 2023-01-27 14:55:00 qty: -340 SHORT  OrderStatus.OPEN 
[2023-10-23 20:31:39.164 __call__] TRADE: 2023-01-27T14:56 NVDA  2023-01-27 14:56:00 qty: -243 prc: 204.96   order: NVDA 2023-01-27 14:55:00 qty: -243 SHORT  OrderStatus.OPEN 
[2023-10-23 20:31:39.164 __call__] TRADE: 2023-01-27T14:56 AAPL  2023-01-27 14:56:00 qty: 340 prc: 146.98   order: AAPL 2023-01-27 14:55:00 qty: 340 LONG  OrderStatus.OPEN 
[2023-10-23 20:31:39.165 __call__] TRADE: 2023-01-27T14:56 NVDA  2023-01-27 14:56:00 qty: 243 prc: 204.96   order: NVDA 2023-01-27 14:55:00 qty: 243 LONG  OrderStatus.OPEN 
[2023-10-23 20:31:39.165 __call__] TRADE: 2023-01-27T14:59 AAPL  2023-01-27 14:59:00 qty: -340 prc: 146.9   order: AAPL 2023-01-27 1

[2023-10-23 20:31:39.183 __call__] TRADE: 2023-01-30T09:40 AAPL  2023-01-30 09:40:00 qty: -344 prc: 144.97   order: AAPL 2023-01-30 09:39:00 qty: -344 SHORT  OrderStatus.OPEN 
[2023-10-23 20:31:39.183 __call__] TRADE: 2023-01-30T09:40 NVDA  2023-01-30 09:40:00 qty: -250 prc: 199.04   order: NVDA 2023-01-30 09:39:00 qty: -250 SHORT  OrderStatus.OPEN 
[2023-10-23 20:31:39.184 __call__] TRADE: 2023-01-30T09:41 AAPL  2023-01-30 09:41:00 qty: 344 prc: 144.8   order: AAPL 2023-01-30 09:40:00 qty: 344 LONG  OrderStatus.OPEN 
[2023-10-23 20:31:39.184 __call__] TRADE: 2023-01-30T09:41 NVDA  2023-01-30 09:41:00 qty: 251 prc: 198.63   order: NVDA 2023-01-30 09:40:00 qty: 251 LONG  OrderStatus.OPEN 
[2023-10-23 20:31:39.185 __call__] TRADE: 2023-01-30T09:41 AAPL  2023-01-30 09:41:00 qty: -344 prc: 144.8   order: AAPL 2023-01-30 09:40:00 qty: -344 SHORT  OrderStatus.OPEN 
[2023-10-23 20:31:39.185 __call__] TRADE: 2023-01-30T09:41 NVDA  2023-01-30 09:41:00 qty: -251 prc: 198.63   order: NVDA 2023-01

[2023-10-23 20:31:39.203 __call__] TRADE: 2023-01-30T10:01 NVDA  2023-01-30 10:01:00 qty: 251 prc: 199.655   order: NVDA 2023-01-30 10:00:00 qty: 251 LONG  OrderStatus.OPEN 
[2023-10-23 20:31:39.204 __call__] TRADE: 2023-01-30T10:01 AAPL  2023-01-30 10:01:00 qty: -344 prc: 144.925   order: AAPL 2023-01-30 10:00:00 qty: -344 SHORT  OrderStatus.OPEN 
[2023-10-23 20:31:39.204 __call__] TRADE: 2023-01-30T10:01 NVDA  2023-01-30 10:01:00 qty: -251 prc: 199.655   order: NVDA 2023-01-30 10:00:00 qty: -251 SHORT  OrderStatus.OPEN 
[2023-10-23 20:31:39.205 __call__] TRADE: 2023-01-30T10:02 AAPL  2023-01-30 10:02:00 qty: 345 prc: 144.95   order: AAPL 2023-01-30 10:01:00 qty: 345 LONG  OrderStatus.OPEN 
[2023-10-23 20:31:39.205 __call__] TRADE: 2023-01-30T10:02 NVDA  2023-01-30 10:02:00 qty: 250 prc: 199.285   order: NVDA 2023-01-30 10:01:00 qty: 250 LONG  OrderStatus.OPEN 
[2023-10-23 20:31:39.205 __call__] TRADE: 2023-01-30T10:02 AAPL  2023-01-30 10:02:00 qty: -345 prc: 144.95   order: AAPL 2023

[2023-10-23 20:31:39.223 __call__] TRADE: 2023-01-30T10:13 AAPL  2023-01-30 10:13:00 qty: 345 prc: 144.539   order: AAPL 2023-01-30 10:12:00 qty: 345 LONG  OrderStatus.OPEN 
[2023-10-23 20:31:39.224 __call__] TRADE: 2023-01-30T10:13 NVDA  2023-01-30 10:13:00 qty: 251 prc: 198.52   order: NVDA 2023-01-30 10:12:00 qty: 251 LONG  OrderStatus.OPEN 
[2023-10-23 20:31:39.224 __call__] TRADE: 2023-01-30T10:13 AAPL  2023-01-30 10:13:00 qty: -345 prc: 144.539   order: AAPL 2023-01-30 10:12:00 qty: -345 SHORT  OrderStatus.OPEN 
[2023-10-23 20:31:39.224 __call__] TRADE: 2023-01-30T10:13 NVDA  2023-01-30 10:13:00 qty: -251 prc: 198.52   order: NVDA 2023-01-30 10:12:00 qty: -251 SHORT  OrderStatus.OPEN 
[2023-10-23 20:31:39.225 __call__] TRADE: 2023-01-30T10:14 AAPL  2023-01-30 10:14:00 qty: 345 prc: 144.6   order: AAPL 2023-01-30 10:13:00 qty: 345 LONG  OrderStatus.OPEN 
[2023-10-23 20:31:39.226 __call__] TRADE: 2023-01-30T10:14 NVDA  2023-01-30 10:14:00 qty: 251 prc: 199.14   order: NVDA 2023-01-

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Lets run the strategy, plot the results and look at the returns

In [4]:
strategy.df_roundtrip_trades()

,symbol,multiplier,entry_timestamp,exit_timestamp,qty,entry_price,exit_price,entry_reason,exit_reason,entry_commission,exit_commission,net_pnl
0,AAPL,1,2023-01-04 09:32:00,2023-01-04 09:32:00,-395,126.7,126.7,SHORT,LONG,0,0,-0
1,NVDA,1,2023-01-04 09:32:00,2023-01-04 09:32:00,-343,145.43999,145.43999,SHORT,LONG,0,0,-0
2,AAPL,1,2023-01-04 09:33:00,2023-01-04 09:33:00,-394,126.3201,126.3201,SHORT,LONG,0,0,-0
3,NVDA,1,2023-01-04 09:33:00,2023-01-04 09:33:00,-343,145.2,145.2,SHORT,LONG,0,0,-0
4,AAPL,1,2023-01-04 09:34:00,2023-01-04 09:34:00,-395,126.71999,126.71999,SHORT,LONG,0,0,-0
...,...,...,...,...,...,...,...,...,...,...,...,...
8529,NVDA,1,2023-01-30 15:58:00,2023-01-30 15:58:00,261,191.545,191.545,LONG,SHORT,0,0,0
8530,AAPL,1,2023-01-30 15:59:00,2023-01-30 15:59:00,349,143.03,143.03,LONG,SHORT,0,0,0
8531,NVDA,1,2023-01-30 15:59:00,2023-01-30 15:59:00,261,191.61,191.61,LONG,SHORT,0,0,0
8532,AAPL,1,2023-01-30 16:00:00,2023-01-30 16:00:00,349,143,143,LONG,SHORT,0,0,0


In [ ]:
strategy.evaluate_returns();